# Dataloader

In [1]:
import os
from PIL import Image, UnidentifiedImageError
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder
from torchvision import transforms
from torchvision.models import resnet18, ResNet18_Weights
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from tqdm import tqdm


In [ ]:
# Settings -> Developer Settings -> Personal Access Tokens -> Token (classic)
os.environ['GITHUB_TOKEN'] = ""

GITHUB_USERNAME = "Codfishz"
REPO_NAME       = "ASR"
TOKEN = os.environ.get("GITHUB_TOKEN")
repo_url        = f"https://{TOKEN}@github.com/{GITHUB_USERNAME}/{REPO_NAME}.git"
!git clone {repo_url}

Cloning into 'ASR'...
remote: Enumerating objects: 89, done.
remote: Counting objects: 100% (89/89), done.
remote: Compressing objects: 100% (68/68), done.
remote: Total 89 (delta 43), reused 62 (delta 20), pack-reused 0 (from 0)
Receiving objects: 100% (89/89), 12.26 MiB | 17.27 MiB/s, done.
Resolving deltas: 100% (43/43), done.


In [3]:
!cd {REPO_NAME} && git pull

Already up to date.


In [4]:
os.chdir('ASR/Script')

In [5]:
%run "Datapreparation_YOLO.ipynb"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 kB 14.0 MB/s eta 0:00:00
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.7.4.2
    Uninstalling kaggle-1.7.4.2:
      Successfully uninstalled kaggle-1.7.4.2
Dataset URL: https://www.kaggle.com/datasets/nirmalsankalana/crop-pest-and-disease-detection
License(s): CC0-1.0
✅ Total images found in dataset: 25220
Images before filter: 25220


Saving train: 100%|██████████| 20176/20176 [00:31<00:00, 635.58it/s]



train split summary:
  Total images: 20093



Saving val: 100%|██████████| 2522/2522 [00:03<00:00, 647.05it/s]



val split summary:
  Total images: 2514



Saving test: 100%|██████████| 2522/2522 [00:03<00:00, 649.63it/s]


test split summary:
  Total images: 2519

  Images after filter : 25126



In [6]:
# Categories
categories = sorted([d for d in os.listdir(original_base) if os.path.isdir(os.path.join(original_base, d))])

# Set image path and labels
image_paths = []
image_labels = []

base_path = "/content/data"

for category in categories:
    category_dir = os.path.join(base_path, category)
    filenames = [f for f in os.listdir(category_dir) if f.endswith(".jpg")]
    for filename in tqdm(filenames, desc=f"Processing '{category}'"):
      image_path = os.path.join(category_dir, filename)
      try:
          img = Image.open(image_path)
          img = img.convert("RGB")
          img.load()  # load image each time to remove the damaged image thoroughly
          image_paths.append(image_path)
          image_labels.append(category)
      except (UnidentifiedImageError, OSError):
          continue

# Label encoder
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(image_labels)

# Dataset
class CropDiseaseDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert("RGB")
        if self.transform:
            image = self.transform(image)
        label = self.labels[idx]
        return image, label

# Augmentation
transform = transforms.Compose([
    transforms.Resize((224, 224)), # To fit the pretrain model (说是resnet的官方推荐输入大小)
    # transforms.RandomRotation(20),
    # transforms.ColorJitter(brightness=0.1, contrast=0.1),
    # transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # To fit the pretrain model (没有很懂啊，加上之后图片群魔乱舞)
])


# Make Dataset and DataLoader
dataset = CropDiseaseDataset(image_paths, encoded_labels, transform=transform)
dataloader = DataLoader(dataset, batch_size=256, shuffle=True, num_workers=0, pin_memory=True)

# Visualization
# plt.figure(figsize=(10, 10))
# for images, labels in dataloader:
#     for i in range(9):
#         img = images[i].permute(1, 2, 0).numpy()
#         plt.subplot(3, 3, i+1)
#         plt.imshow(img)
#         plt.axis('off')
#     break
# plt.show()


Processing 'Tomato verticulium wilt': 100%|██████████| 773/773 [00:00<00:00, 786.91it/s]


In [7]:
print(len(dataset))

25126


# Feature extraction (ResNet18)

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load pretrained ResNet
weights = ResNet18_Weights.DEFAULT
resnet18 = resnet18(weights=weights)

# Remove the final fully connected layer
feature_extractor = nn.Sequential(*list(resnet18.children())[:-1])
feature_extractor.to(device)
feature_extractor.eval()

# Extract features

def extract_features(dataloader, model, device):
    all_features = []
    all_labels = []

    with torch.no_grad():
        for images, labels in tqdm(dataloader, desc="Extracting features"):
            images = images.to(device)
            features = model(images)                  # shape: [B, 512, 1, 1]
            features = features.view(features.size(0), -1)  # flatten to [B, 512]
            all_features.append(features.cpu())
            all_labels.append(labels)

    return torch.cat(all_features), torch.cat(all_labels)

features, labels = extract_features(dataloader, feature_extractor, device)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 234MB/s]
Extracting features: 100%|██████████| 99/99 [01:37<00:00,  1.01it/s]


In [9]:
# Results
print("Features shape:", features.shape)  # [N, 512]
print("Labels shape:", labels.shape)      # [N]

Features shape: torch.Size([25126, 512])
Labels shape: torch.Size([25126])


In [10]:
from scipy.cluster.hierarchy import linkage, to_tree

# features: numpy array, shape (N, D)
Z = linkage(features, method='ward')
root, nodes_list = to_tree(Z, rd=True)

pruning = [root]


In [11]:
import random
def select_dh_cluster(pruning, labeled_indices, labels, alpha=0.05):
    """
    pruning: 当前的节点列表
    labeled_indices: dict，从样本 idx 到标签
    labels: 完整标签数组，用于获取 ground truth
    """
    best_score, best_node = -1, None
    z = scipy.stats.norm.ppf(1 - alpha/2)
    for node in pruning:
        leaves = node.get_leaves()
        # 找出这个节点下已经标注的样本
        L = [labeled_indices[i] for i in leaves if i in labeled_indices]
        if len(L)==0:
            score = len(leaves)  # 无信息时优先大节点
        else:
            # 多数标签频率
            p_hat = max(L.count(c)/len(L) for c in set(L))
            n = len(L)
            # 置信下界
            lower = p_hat - z*(p_hat*(1-p_hat)/n)**0.5
            eps = 1 - max(lower, 0)
            score = eps * len(leaves)
        if score > best_score:
            best_score, best_node = score, node
    # 从 best_node 的 leaves 中随机抽一个未标记样本
    candidates = [i for i in best_node.get_leaves() if i not in labeled_indices]
    sample_idx = random.choice(candidates)
    return sample_idx, best_node


In [12]:
def update_pruning(pruning, sampled_node, labeled_indices, alpha=0.05):
    new_pruning = []
    z = scipy.stats.norm.ppf(1 - alpha/2)
    for node in pruning:
        if node is sampled_node:
            # 计算该节点下标注后多数标签频率和置信区间
            leaves = node.get_leaves()
            L = [labeled_indices[i] for i in leaves if i in labeled_indices]
            p_hat = max(L.count(c)/len(L) for c in set(L))
            n = len(L)
            lower = p_hat - z*(p_hat*(1-p_hat)/n)**0.5
            eps = 1 - lower
            # 如果纯度不够（eps 较大），则拆分
            if eps > some_threshold:   # e.g., eps > 0.1
                new_pruning.extend([node.get_left(), node.get_right()])
                continue
        new_pruning.append(node)
    return new_pruning


In [24]:
def get_leaves(node):
    # Leaf node: left/right 都是 None
    if node.left is None and node.right is None:
        return [node.id]
    leaves = []
    if node.left:
        leaves += get_leaves(node.left)
    if node.right:
        leaves += get_leaves(node.right)
    return leaves

In [13]:
!pip install ultralytics
!pip install torchinfo

import ultralytics
ultralytics.checks()


Ultralytics 8.3.111 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
Setup complete ✅ (12 CPUs, 53.0 GB RAM, 43.7/235.7 GB disk)


# Functions

In [17]:
def setup_active_dataset():

    orig_train_dir = os.path.join(ORIGINAL_DATASET_DIR, TRAIN_SUBDIR)
    active_train_dir = os.path.join(ACTIVE_DATASET_DIR, TRAIN_SUBDIR)
    os.makedirs(active_train_dir, exist_ok=True)

    for cls in os.listdir(orig_train_dir):
        cls_path = os.path.join(orig_train_dir, cls)
        if os.path.isdir(cls_path):
            os.makedirs(os.path.join(active_train_dir, cls), exist_ok=True)
    print(f"Complete creating train_dir {active_train_dir}")

    orig_val_dir = os.path.join(ORIGINAL_DATASET_DIR, VAL_SUBDIR)
    active_val_dir = os.path.join(ACTIVE_DATASET_DIR, VAL_SUBDIR)
    if os.path.exists(orig_val_dir):
        if os.path.exists(active_val_dir):
            shutil.rmtree(active_val_dir)
        shutil.copytree(orig_val_dir, active_val_dir)

    orig_test_dir = os.path.join(ORIGINAL_DATASET_DIR, TEST_SUBDIR)
    active_test_dir = os.path.join(ACTIVE_DATASET_DIR, TEST_SUBDIR)
    if os.path.exists(orig_test_dir):
        if os.path.exists(active_test_dir):
            shutil.rmtree(active_test_dir)
        shutil.copytree(orig_test_dir, active_test_dir)

In [18]:
def get_all_samples():

    samples = []
    orig_train_dir = os.path.join(ORIGINAL_DATASET_DIR, TRAIN_SUBDIR)
    for cls in os.listdir(orig_train_dir):
        cls_path = os.path.join(orig_train_dir, cls)
        if os.path.isdir(cls_path):
            image_files = [f for f in os.listdir(cls_path) if os.path.isfile(os.path.join(cls_path, f))]
            for f in image_files:
                samples.append((cls, f))
    return samples

In [19]:
import shutil
def copy_samples(sample_list):

    for cls, file_name in sample_list:
        src_path = os.path.join(ORIGINAL_DATASET_DIR, TRAIN_SUBDIR, cls, file_name)
        dst_path = os.path.join(ACTIVE_DATASET_DIR, TRAIN_SUBDIR, cls, file_name)
        if not os.path.exists(dst_path):
            shutil.copy(src_path, dst_path)

In [20]:
import random
def stratified_sample(all_samples, n_initial):

    samples_by_class = {}
    for cls, filename in all_samples:
        samples_by_class.setdefault(cls, []).append((cls, filename))

    stratified = []
    for cls, samples in samples_by_class.items():
        stratified.append(random.choice(samples))
    remaining_count = n_initial - len(stratified)
    if remaining_count > 0:
        remaining_samples = list(set(all_samples) - set(stratified))
        additional_samples = random.sample(remaining_samples, remaining_count)
        stratified.extend(additional_samples)

    return stratified



# Hyperparameters

In [14]:
# Hyperparameters
ORIGINAL_DATASET_DIR = '/content/data_yolo'
ACTIVE_DATASET_DIR = '/content/data_active'

TRAIN_SUBDIR = 'train'
VAL_SUBDIR = 'val'
TEST_SUBDIR = 'test'
Num_Train = 20093
N_INITIAL = int(0.3 * Num_Train)
N_PER_PHASE = int(0.1 * Num_Train)
NUM_PHASES = 5

# DH selection

In [27]:
from ultralytics import YOLO
import random
import os
import csv
from scipy.cluster.hierarchy import linkage, to_tree
import scipy.stats as stats
from torch.utils.data import DataLoader

# --- Helper to collect leaf indices from a ClusterNode ---
def get_leaves(node):
    if node.left is None and node.right is None:
        return [node.id]
    leaves = []
    if node.left is not None:
        leaves += get_leaves(node.left)
    if node.right is not None:
        leaves += get_leaves(node.right)
    return leaves

# --- 1) 构建/初始化 Active 数据集 & 提取 all_samples 列表 ---
setup_active_dataset()
all_samples = get_all_samples()

# --- 2) 初始分层随机采样 & 拷贝到 active_data ---
current_sample_list = stratified_sample(all_samples, N_INITIAL)
copy_samples(current_sample_list)

# --- 3) 全池特征提取 & 构建层次聚类树 ---
# 构造完整 Dataset & DataLoader
full_paths  = [os.path.join(ORIGINAL_DATASET_DIR, TRAIN_SUBDIR, cls, fn)
               for cls, fn in all_samples]
full_labels = [label for label in encoded_labels]
full_dataset = CropDiseaseDataset(full_paths, full_labels, transform=transform)
full_loader  = DataLoader(full_dataset, batch_size=256, shuffle=False, num_workers=4)

# 提取特征
full_features, full_labels = extract_features(full_loader, feature_extractor, device)
features_np  = full_features.numpy()  # shape: (N, D)
labels_np    = full_labels.numpy()    # shape: (N,)

# 用 Ward 方法做层次聚类
Z = linkage(features_np, method='ward')
root, _ = to_tree(Z, rd=True)

# 初始化 DH 的 pruning 列表和标注字典
pruning = [root]
labeled_indices = {}
for cls, fn in current_sample_list:
    idx = all_samples.index((cls, fn))
    labeled_indices[idx] = labels_np[idx]

# --- 4) 首轮 YOLO 训练 & 日志初始化 ---
log_path = "accuracy_log_dh.csv"
with open(log_path, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["Phase", "Num_Images", "Top-1", "Top-5"])

print("Initial Epoch")
model = YOLO('yolo11n-cls.pt')
results = model.train(
    data=ACTIVE_DATASET_DIR,
    epochs=1, imgsz=640,
    name="active-phase0", project="runs/classify"
)
with open(log_path, "a", newline="") as f:
    writer = csv.writer(f)
    writer.writerow([0, len(current_sample_list), results.top1, results.top5])

# 构造 remaining_samples 池
remaining_samples = list(set(all_samples) - set(current_sample_list))

# --- 5) DH 主循环 ---
alpha = 0.05
z = stats.norm.ppf(1 - alpha / 2)
for phase in range(1, NUM_PHASES):
    if not remaining_samples:
        print("No more samples")
        break

    # (a) 在 pruning 中选出得分最高的节点
    best_score, best_node = -1, None
    for node in pruning:
        leaves = get_leaves(node)
        L = [labeled_indices[i] for i in leaves if i in labeled_indices]
        if not L:
            score = len(leaves)
        else:
            p_hat = max(L.count(c) / len(L) for c in set(L))
            n = len(L)
            lower = p_hat - z * (p_hat * (1 - p_hat) / n) ** 0.5
            eps = 1 - max(lower, 0)
            score = eps * len(leaves)
        if score > best_score:
            best_score, best_node = score, node

    # (b) 在 best_node 下随机选一个未标注样本
    candidates = [i for i in get_leaves(best_node) if i not in labeled_indices]
    sample_idx = random.choice(candidates)
    cls, fn = all_samples[sample_idx]
    copy_samples([(cls, fn)])                  # 拷贝单张图片
    labeled_indices[sample_idx] = labels_np[sample_idx]
    current_sample_list.append((cls, fn))
    remaining_samples.remove((cls, fn))

    # (c) 更新 pruning：若纯度不够则拆分该节点
    leaves = get_leaves(best_node)
    L = [labeled_indices[i] for i in leaves if i in labeled_indices]
    if L:
        p_hat = max(L.count(c) / len(L) for c in set(L))
        lower = p_hat - z * (p_hat * (1 - p_hat) / len(L)) ** 0.5
        eps = 1 - max(lower, 0)
    else:
        eps = 1.0
    if eps > 0.1 and best_node.left and best_node.right:
        pruning.remove(best_node)
        pruning.extend([best_node.left, best_node.right])

    print(f"[DH Phase {phase}] added 1 sample, total={len(current_sample_list)}")

    # (d) 用 YOLO 在累积样本上微调并记录准确率
    model = YOLO(f"runs/classify/active-phase{phase-1}/weights/last.pt")
    results = model.train(
        data=ACTIVE_DATASET_DIR,
        epochs=1, imgsz=640,
        name=f"active-phase{phase}", project="runs/classify"
    )
    with open(log_path, "a", newline="") as f:
        writer = csv.writer(f)
        writer.writerow([phase, len(current_sample_list), results.top1, results.top5])

    print(f"Phase {phase} training completed")


Complete creating train_dir /content/data_active/train


Extracting features: 100%|██████████| 79/79 [00:20<00:00,  3.91it/s]


Initial Epoch
Ultralytics 8.3.111 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
engine/trainer: task=classify, mode=train, model=yolo11n-cls.pt, data=/content/data_active, epochs=1, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=runs/classify, name=active-phase04, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=Tr

train: Scanning /content/data_active/train... 15923 images, 0 corrupt: 100%|██████████| 15923/15923 [00:04<00:00, 3475.27it/s]

train: New cache created: /content/data_active/train.cache


...

val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 623.2±392.3 MB/s, size: 66.7 KB)


val: Scanning /content/data_active/val... 2514 images, 0 corrupt: 100%|██████████| 2514/2514 [00:00<?, ?it/s]


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000385, momentum=0.9) with parameter groups 39 weight(decay=0.0), 40 weight(decay=0.0005), 40 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/classify/active-phase2
Starting training for 1 epochs...

      Epoch    GPU_mem       loss  Instances       Size


        1/1      1.79G     0.6678         16        640:   3%|▎         | 28/996 [00:04<02:19,  6.95it/s]


KeyboardInterrupt: 